# Monte Carlo simulation to obtain the global minimum

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/develop/notebook/statistical-mechanics/monte_carlo_parabolic.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**

<p style="text-align: justify;font-size:15px">
    Search global minimum is a common computational problem. In 
    computational materials science, we need to search global minimum 
    for the most stable configuration. It is popular to use Monte Carlo 
    simulations to find the global minimum. We demonstrate that using Monte 
    Carlo simulations find the global minimum of a parabolic potential well 
    in this notebook.
</p>

<details close>
    <summary style="font-size: 20px"><b>Sub-goals</b></summary>
    <ol style="text-align: justify;font-size:15px">
        <li> Understand the concept of the potential energy surface.</li>
        <li> Learn the algorithm of the Metropolis Monte Caro simulation.</li>
        <li> Examine the results for various parameters (temperature, 
            max move size and numbers of steps).</li>
    </ol>
</details>

## **Background theory**

<p style="text-align: justify;font-size:15px">
    Monte Carlo methods are a collection of numerical methods by employing 
    random numbers. Practically, we employed the Metropolis-Hastings 
    algorithm to get the global minimum of the potential energy 
    surface in this notebook.    
</p>

<details close>
<summary style="font-size: 20px">Markov chain Monte Carlo (MCMC)</summary>
    
<p style="text-align: justify;font-size:15px">
    A Markov chain is a stochastic model that describes a sequence of
    possible events, where the probability of each event only depends
    on the state obtained in the previous event. A diagram representing
    a two-state Markov process is showing in the figure below:
</p>

<div class="container" style="text-align: center; width: 300px;">
  <img src="images/Markov_chain.png" alt="Markov chain" class="image">
  <div class="overlay">A two state Markov chain (figure from Wikipedia).</div>
</div>
    
<p style="text-align: justify;font-size:15px">
    In the figure, two states are labelled as A and E. Each number 
    represents the probability of the Markov process changing from one 
    state to another state, where the arrow indicates the direction. 
    Markov chain Monte Carlo (MCMC) is a numerical method to sample by 
    using the Markov chain. MCMC method can lead to fast converge from 
    initial state to stable state. It is a popular method to solve 
    computational chemistry and physics problems.
</p>
    
</details>

<details close>
<summary style="font-size: 20px">Metropolis-Hastings algorithm</summary>
    
<p style="text-align: justify;font-size:15px">
    Metropolis-Hastings algorithm is a statistical method for the MCMC. 
    It is used to obtain a sequence of random samples from a probability 
    distribution, which is difficult to get from direct sampling. This 
    sequence can be used to approximate the distribution or to calculate 
    an integral, for example, an expected value.
</p>
    
<p style="text-align: justify;font-size:15px">
    The algorithm of the Metropolis-Hastings is presented as shown below:
</p>
    
<p style="text-align: justify;font-size:15px">
<dl>
    <dt><b>Initialization</b></dt>
      <dd>- choose the initial state x$_0$.</dd>
      <dd>- t = 0.</dd>
    <dt><b>Iteration</b></dt>
      <dd>- Generate a new state x' from the distribution $Q(x'|x_t)$.</dd>
      <dd>- Compute the probability of the new state $A(x'|x)=min(1, \frac{P(x')}{P(x)}\frac{Q(x|x')}{Q(x'|x)})$.</dd>
      <dd>- Generate a random number $\mu$ from a uniform distribution between [0, 1].</dd>
      <dd>- If $\mu \leq A(x'|x)$, accept the new state and $x_{t+1} = x'$.</dd>
      <dd>- If $\mu > A(x'|x)$, reject the new state and store the old state $x_{t+1} = x_t$.</dd>
      <dd>- Update step $t=t+1$.</dd>
</dl>
</p>

</details>

## **Tasks and exercises**

<ol style="text-align: justify;font-size:15px">
    <li> Could you report the global minimum (x, y coordinations) from the top left 3D plot?
    <details style="color: blue">
    <summary>Hints</summary>
        Choose the potential from the dropdown list. You will find only one global 
        minimum at (x=0, y=0) in the one parabolic well case. Otherwise, two global 
        minimums locate at around (x=0, y=3.13) and (x=0, y=-3.13) in the two 
        parabolic wells potential.
    </details>
    </li>
    <li> How to generate the next step in this simulation?
    <details style="color: blue">
    <summary>Hints</summary>
        The next step is generating random walks on the potential energy surface. 
        Two random numbers are generated from a uniform distribution between 
        [0, max_move]. The "max_move" value is determined from the "Max move" slider.
    </details>
    </li>
    <li> How to compute the acceptance ratio $\frac{P(x')}{P(x)}$?
    <details style="color: blue">
    <summary>Hints</summary>
        For particle system, it always follows the Boltzmann distribution. Hence,
        the acceptance ratio is $\frac{e^{-\frac{E'}{kT}}}{e^{-\frac{E}{kT}}}=
        e^{-\frac{E'-E}{kT}}$. So we need to compute the energy change $\Delta E = E' -E$
        for each step.
    </details>
    </li>
</ol>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
import math
from plotly.subplots import make_subplots
from ipywidgets import FloatSlider, IntSlider, FloatLogSlider, Button
from ipywidgets import Output, HBox, VBox, IntProgress, Dropdown, Layout, Label

In [ ]:
#all the widget components

temp_slider = FloatSlider(min=0.1, max=5, value=0.65, continuous_update=False, description="Temperature")
move_size_slider = FloatLogSlider(min=-1, max=1, value=0.25, continuous_update=False, description="Max move size")
num_iterations_slider = IntSlider(min=100000, max=1000000, value=100000, continuous_update=False, description="Num iterations")
run_button = Button(description="Run Monte-Carlo")
run_button.style.button_color = 'green'
traces_button = Button(description="Show Traces")
x_slider = FloatSlider(value=9.0, min=-10.0, max=10.0, continuous_update=False, description="Initial pos. x")
y_slider = FloatSlider(value=9.0, min=-10.0, max=10.0, continuous_update=False, description="Initial pos. y")
potential_dropdown = Dropdown(options=[('one parabolic well',1), ('two parabolic wells',2)],
                              value = 1, description='Potential:', layout=Layout(width='250px'))
label1 = Label(value='(only shows 300 steps from start to end of the simulations.)')

In [ ]:
A1 = 1. # potential curvature first axis
A2 = 1.e-1 # potential curvature second axis

# Starting point of our MC, for the two coordinates
starting_x1 = 9.0
starting_x2 = 9.0

def get_energy(x, y):
    """Compute the potential energy according to the dropbox."""
    if potential_dropdown.value == 1:
        return 0.5 * x**2 + 0.5 * 4.0 * y**2
    elif potential_dropdown.value == 2:
        return 0.5 * A1 * x**2 + 0.5 * A2 * (y**4 - 20 * y**2)

def get_property(x1, x2):
    # I can do something else, I decide to sample the energy of one of the two coordinates
    #return 0.5 * A2 * (x2**4 - 4 * x2**2)
    
    # I just return the second coordinate, to see its average
    return x2

In [ ]:
x, y = np.mgrid[-10:10:100j, -10:10:100j]
z = get_energy(x, y)

# figure one to show the potential energy surface and current position
fig1 = go.FigureWidget(data=[go.Surface(z=z, x=x, y=y, opacity=0.5),
                             go.Scatter3d(x=[starting_x1], 
                                          y=[starting_x2], 
                                          z=[get_energy(starting_x1, starting_x2)], 
                                          mode='markers+text',
                                          text=['Current position'],
                                          marker=dict(size=8, color='red'))])

def on_potential_change(c):
    """Callback function for the change of the dropdown."""
    fig1.data[0].z = get_energy(x, y)
    fig1.data[1].x = [x_slider.value]
    fig1.data[1].y = [y_slider.value]
    fig1.data[1].z = [get_energy(x_slider.value, y_slider.value)]
    
potential_dropdown.observe(on_potential_change)

In [ ]:
# figure 2 is the historgram of the coordinations in heatmap.
fig2 = go.FigureWidget()

fig2.add_trace(go.Scatter(
    x=[starting_x1],
    y=[starting_x2],
    mode='lines+markers',
    visible = False,
    showlegend=False,
    marker=dict(
        symbol='circle',
        opacity=0.7,
        color='white',
        size=5,
        line=dict(width=1),
    ),
))

fig2.add_trace(go.Histogram2d(
    x=[],
    y=[],
    histnorm='probability',
    autobinx = False,
    xbins=dict(start=-10, end=10, size=0.1),
    autobiny = False,
    ybins=dict(start=-10, end=10, size=0.1),
    colorscale=[[0, 'rgb(12,51,131)'], [0.25, 'rgb(10,136,186)'], [0.6, 'rgb(242,211,56)'], 
                [0.75, 'rgb(242,143,56)'], [1, 'rgb(217,30,30)']]
))

fig1.update_layout(title='Potential energy surface', autosize=False,
                  width=420, height=420,
                  margin=dict(l=10, r=10, b=30, t=30))

fig2.update_layout(title='Histogram of the coordinations',
    xaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
    yaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
    xaxis_title="X",
    yaxis_title="Y",
    autosize=False,
    height=420,
    width=430,
    margin=dict(l=30, r=30, b=60, t=60),
    hovermode='closest',

)

fig3 = go.FigureWidget()

fig3.add_trace(go.Scatter(x=[], y=[], mode='lines',
    name='Total energy',
    line=dict(color='red', width=2),
    connectgaps=True,
))

fig3.update_layout(
    height = 200,
    width = 530,
    xaxis_title = 'Monte-Carlo move',
    yaxis_title = 'Total energy',
    margin=dict(l=30, r=30, b=10, t=10),
)

display(HBox([fig1, fig2]))

In [ ]:
def run_mc(move_size, num_iterations, temp):
    """
    :param move_size: max move size for the two axes x1 and x2.
        In principle, the move size could/should be different for x1 and x2, here I choose the same for simplicity!
        
    :param num_iterations: total number of Monte-Carlo iterations
    """
    x1 = starting_x1
    x2 = starting_x2
    energy = get_energy(x1, x2)

    all_x1 = [x1]
    all_x2 = [x2]
    all_properties = [get_property(x1, x2)]
    all_energies = [get_energy(x1, x2)]
    count_accepted = 0
    count_refused = 0

    for iter_cnt in range(num_iterations):
        # shift by a random value between -move_size and + move_size
        new_x1 = x1 + ((random.random() - 0.5) * 2) * move_size
        new_x2 = x2 + ((random.random() - 0.5) * 2) * move_size
        new_energy = get_energy(new_x1, new_x2)

        if new_energy < energy:
            accepted = True
        else:
            # boltzmann_k = 1 in these units
            probability = math.exp(-(new_energy - energy)/temp)
            # random.random() is a random number between 0 and 1
            # also probability is between 0 and 1
            # so if I accept only if random.random() is < probability,
            # I'm accepting the move with 'probability' probability
            accepted = random.random() < probability

        if accepted:
            x1 = new_x1
            x2 = new_x2
            energy = new_energy
            count_accepted += 1
        else:
            count_refused += 1
        all_properties.append(get_property(x1, x2))
        all_energies.append(energy)
        all_x1.append(x1)
        all_x2.append(x2)
        
    average_integrated_energy = np.cumsum(all_energies) / (1 + np.arange(len(all_energies)))
    average_property = np.cumsum(all_properties) / (1 + np.arange(len(all_properties)))

    return count_accepted, count_refused, average_integrated_energy, average_property, all_x1, all_x2

In [ ]:
controls = VBox([x_slider, y_slider, potential_dropdown, temp_slider, move_size_slider, num_iterations_slider])

display(HBox([controls, fig3]))
display(run_button, HBox([traces_button, label1]))

In [ ]:
def change_init_position(c):
    """Callback function for the change of the x, y sliders."""
    global starting_x1, starting_x2
    starting_x1 = x_slider.value
    starting_x2 = y_slider.value
    fig1.data[1].x = [x_slider.value]
    fig1.data[1].y = [y_slider.value]
    fig1.data[1].z = [get_energy(x_slider.value, y_slider.value)]
    
x_slider.observe(change_init_position, names='value')
y_slider.observe(change_init_position, names='value')

In [ ]:
def interactive_plot(button):
    """Callback function for the run button."""
    global all_x1, all_x2
    temp = temp_slider.value
    move_size=move_size_slider.value
    num_iterations=num_iterations_slider.value
    
    run_button.disabled = True
    run_button.style.button_color = 'red'
    run_button.description = "Runing..."
    traces_button.disabled = True
    try:
        count_accepted, count_refused, average_integrated_energy, average_property, all_x1, all_x2 = run_mc(
            move_size=move_size, num_iterations=num_iterations, temp=temp)
        
        fig1.data[1].x = [all_x1[-1]]
        fig1.data[1].y = [all_x2[-1]]
        fig1.data[1].z = [get_energy(all_x1[-1], all_x2[-1])]
    
        snum = int(len(all_x1)/300)
        fig2.data[0].x = all_x1[::snum]
        fig2.data[0].y = all_x2[::snum]
        
        fig2.data[1].x = all_x1
        fig2.data[1].y = all_x2
        
        skip_start = 10000
        
        fig3.data[0].x = np.arange(len(average_integrated_energy) - skip_start)
        fig3.data[0].y = average_integrated_energy[skip_start:]
    finally:
        run_button.disabled = False
        traces_button.disabled = False
        run_button.style.button_color = 'green'
        run_button.description="Run Monte-Carlo"
        
run_button.on_click(interactive_plot)

In [ ]:
def toggle_show_traces(button):
    """Callback function for the show traces button."""
    fig2.data[0].visible ^= True
    if fig2.data[0].visible:
        traces_button.description = "Hide traces"
    else:
        traces_button.description = "Show traces"
    
traces_button.on_click(toggle_show_traces)


def show_current_point(trace, points, selector):
    for i in points.point_inds:
        fig1.data[1].x = [all_x1[i]]
        fig1.data[1].y = [all_x2[i]]
        fig1.data[1].z = [get_energy(all_x1[i], all_x2[i])]
        
fig3.data[0].on_click(show_current_point)

<details>
    <summary style="font-size: 22px;"><b>Legend</b></summary>

<p style="text-align: justify;font-size:15px">
    The left panel shows the potential energy surface (PES). You 
    can rotate the surface by clicking and dragging. The Z value 
    and colour bar indicate the value of the energy. When the mouse 
    is on the potential energy surface, you will also see a black 
    isoline, which you can use to find the global minimum.
</p>

<p style="text-align: justify;font-size:15px">
    All the control widgets are on the left bottom. You can choose 
    the initial x, y coordinations, the type of the potentials, 
    temperature, max move and number of the interactions. Start and 
    run the MC simulation by clicking the "Run-Monte Carlo" button.
</p>
    
<p style="text-align: justify;font-size:15px">
    The histogram of the x, y coordinations will be shown as a heatmap 
    on the top right after the running of the simulations. By clicking 
    the "Show traces" button, it will show 300 points (evenly distributed) 
    on the heatmap from the Monte Carlo simulation. The total energy as a 
    function of the step is shown on the bottom right. 10000 steps from the 
    beginning of the simulation are skipped for the total energy figure.
</p>
    
</details>